In [20]:
import cv2
import numpy as np

# Get the screen resolution
screen_width, screen_height = 1920, 1080  # Adjust these values according to your screen resolution

# Load the pre-trained face detection cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open a connection to the camera (0 for webcam, or specify the camera index if you have multiple cameras)
cap = cv2.VideoCapture(0)

# Set camera resolution
cap.set(cv2.CAP_PROP_FRAME_WIDTH, screen_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, screen_height)

# Define a dictionary mapping skin types to recommended products from different companies
product_recommendations = {
    'Normal': {
        'Nivea': 'Moisturizer',
        'Mamaearth': 'Hydrating Serum',
        'Ponds': 'Daily Face Cream',
        'ayurvedic solution': 'Aloe Vera Gel'
    },
    'Oily': {
        'Mamaearth': 'Oil-Free Cleanser',
        'Plum Green Tea': 'Mattifying Moisturizer',
        'Ponds': 'Balancing Gel',
        'ayurvedic solution': 'Neem Face Pack'
    },
    'Dry': {
        'Rose': 'Hydrating Cream',
        'Ponds': 'Rich Moisture Balm',
        'Good Vibes': 'Nourishing Face Oil',
        'ayurvedic solution': 'Sandalwood Face Pack'
    }
}

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Skin color detection
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 48, 80], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask_skin = cv2.inRange(hsv_frame, lower_skin, upper_skin)
    skin = cv2.bitwise_and(frame, frame, mask=mask_skin)

    # Check the percentage of skin in the frame
    skin_pixels = np.sum(mask_skin == 255)
    total_pixels = mask_skin.shape[0] * mask_skin.shape[1]
    skin_percentage = (skin_pixels / total_pixels) * 100

    # Determine skin type based on skin percentage
    if skin_percentage < 5:
        skin_type = 'Dry'
    elif skin_percentage < 15:
        skin_type = 'Normal'
    else:
        skin_type = 'Oily'

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        # Enlarge the face covering window
        x -= 30
        y -= 30
        w += 60
        h += 60
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Display skin type and product recommendations from different companies
        cv2.putText(frame, f'Skin Type: {skin_type}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'Skin Percentage: {skin_percentage:.2f}%', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, 'Product Recommendations:', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        y_offset = 120
        for company, product in product_recommendations[skin_type].items():
            cv2.putText(frame, f'{company}: {product}', (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            y_offset += 30

    # Display the frame with detected faces and product recommendations
    cv2.imshow('Face Detection', frame)

    # Check for key press and exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
